In [ ]:
try:
    # Check whether the c3 object is defined
    c3
except NameError:
    # Connect to a c3 cluster and create the c3 object
    from c3python import get_c3
    c3 = get_c3('<vanity_url>', '<tenant>', '<tag>')

In [73]:
#try:
#    # Check whether the c3 object is defined
#    c3
#except NameError:
    # Connect to a c3 cluster and create the c3 object
    from c3python import get_c3
    c3 = get_c3('http://192.168.2.44:8080', 'mnistExercise', 'dev')

Username: BA
Password: ········


In [67]:
!conda install -y dill

Solving environment: done

## Package Plan ##

  environment location: /data0/matthew/Software/NCSA/C3/c3-packages/dtiTraining/mnistExample/c3/py-tensorflow_2_1_0_venv

  added / updated specs:
    - dill


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dill-0.3.2                 |     pyh9f0ad1d_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  dill               conda-forge/noarch::dill-0.3.2-pyh9f0ad1d_0

The following packages will be UPDATED:

  openssl                                 1.1.1g-h516909a_0 --> 1.1.1g-h516909a_1



dill-0.3.2           | 59 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [74]:
import tensorflow as tf
import numpy as np
import pandas as pd
import dill

In [75]:
# Define function to create one-hot labels
def make_onehot(data):
    Res = np.zeros((data.size, data.max()+1))
    Res[np.arange(data.size), data.reshape((data.size,))] = 1
    return Res.reshape(data.shape+(data.max()+1,))

make_onehot_serialized = c3.PythonSerialization.serialize(make_onehot)

In [76]:
# Load and 'reshape' datasets
#mnist_test_images = c3.Dataset.toDataset(this=c3.IDXFile.get('MNIST_Test_Images').getDataset(),
#                                         axesAsRow=[0], axesAsColumn=[1,2])
#mnist_test_images.axes = ['source', 'features']
#mnist_test_labels = c3.Dataset.toDataset(this=c3.IDXFile.get('MNIST_Test_Labels').getDataset(),
#                                         axesAsRow=[0], axesAsColumn=[1])
#mnist_test_labels.axes = ['source', 'features']
#mnist_train_images = c3.Dataset.toDataset(this=c3.IDXFile.get('MNIST_Train_Images').getDataset(),
#                                          axesAsRow=[0], axesAsColumn=[1,2])
#mnist_train_images.axes = ['source', 'features']
#mnist_train_labels = c3.Dataset.toDataset(this=c3.IDXFile.get('MNIST_Train_Labels').getDataset(),
#                                          axesAsRow=[0], axesAsColumn=[1])
#mnist_train_labels.axes = ['source', 'features']

mnist_test_images_spec = c3.IDXFile.get('MNIST_Test_Images').getFileSourceSpec()
mnist_test_labels_spec = c3.IDXFile.get('MNIST_Test_Labels').getFileSourceSpecPreprocess(make_onehot_serialized, 'onehot')
mnist_train_images_spec = c3.IDXFile.get('MNIST_Train_Images').getFileSourceSpec()
mnist_train_labels_spec = c3.IDXFile.get('MNIST_Train_Labels').getFileSourceSpec(make_onehot_serialized, 'onehot')

500 - NotClassified - c3.love.util.OsUtil_err2 [7578.969]
message: "Error executing command: /usr/local/share/c3/condaEnvs/mnistExercise/dev/py-idxfile/bin/python /tmp/pythonActionSourceCache7700729396145615776/IDXFile_getFileSourceSpecPreprocess.py
Traceback (most recent call last):
  File "/tmp/pythonActionSourceCache7700729396145615776/IDXFile_getFileSourceSpecPreprocess.py", line 442, in <module>
    _c3_result = getFileSourceSpecPreprocess(this = _c3_inputs.get('this'),serializedPreprocessor = _c3_inputs.get('serializedPreprocessor'),preprocessFuncName = _c3_inputs.get('preprocessFuncName'),enableLocalClientStorage = _c3_inputs.get('enableLocalClientStorage'))
  File "/tmp/pythonActionSourceCache7700729396145615776/IDXFile_getFileSourceSpecPreprocess.py", line 408, in getFileSourceSpecPreprocess
    if type(preprocess_data) is not np.ndarray:
NameError: name 'preprocess_data' is not defined
"
JSON: {"enableLocalClientStorage": true, "preprocessFuncName": "onehot", "serializedPrepr

C3RuntimeException: 500 - NotClassified - c3.love.util.OsUtil_err2 [7578.969]
message: "Error executing command: /usr/local/share/c3/condaEnvs/mnistExercise/dev/py-idxfile/bin/python /tmp/pythonActionSourceCache7700729396145615776/IDXFile_getFileSourceSpecPreprocess.py
Traceback (most recent call last):
  File "/tmp/pythonActionSourceCache7700729396145615776/IDXFile_getFileSourceSpecPreprocess.py", line 442, in <module>
    _c3_result = getFileSourceSpecPreprocess(this = _c3_inputs.get('this'),serializedPreprocessor = _c3_inputs.get('serializedPreprocessor'),preprocessFuncName = _c3_inputs.get('preprocessFuncName'),enableLocalClientStorage = _c3_inputs.get('enableLocalClientStorage'))
  File "/tmp/pythonActionSourceCache7700729396145615776/IDXFile_getFileSourceSpecPreprocess.py", line 408, in getFileSourceSpecPreprocess
    if type(preprocess_data) is not np.ndarray:
NameError: name 'preprocess_data' is not defined
"
JSON: {"enableLocalClientStorage": true, "preprocessFuncName": "oneho

In [8]:
c3.FileSourceSpec.fetch()

c3.FetchResult<FileSourceSpec>(
 objs=c3.Arry<FileSourceSpec>([c3.FileNdArraySourceSpec(
         locationPrefix='http___yann.lecun.com_exdb_mnist_t10k-labels-idx1-ubyte.gz',
         storageFormat='NPY',
         optimalFileSize=64000000,
         enableLocalClientStorage=True,
         id='a110e642-47a8-42e3-8d5f-92114b902845',
         meta=c3.Meta(
                tenantTagId=3,
                tenant='mnistExercise',
                tag='dev',
                created=datetime.datetime(2020, 7, 22, 18, 2, 39, tzinfo=datetime.timezone.utc),
                createdBy='BA',
                updated=datetime.datetime(2020, 7, 22, 18, 2, 39, tzinfo=datetime.timezone.utc),
                updatedBy='BA',
                timestamp=datetime.datetime(2020, 7, 22, 18, 2, 39, tzinfo=datetime.timezone.utc),
                fetchInclude='[]',
                fetchType='FileSourceSpec'),
         version=2,
         typeIdent='FISS:FNDASS',
         location='http___yann.lecun.com_exdb_mnist_t10k

In [13]:
# Produce stream datasets
mnist_test_images = c3.Dataset.fromNdArrayStreamable(sourceSpec=mnist_test_images_spec)
mnist_test_labels = c3.Dataset.fromNdArrayStreamable(sourceSpec=mnist_test_labels_spec)
mnist_train_images = c3.Dataset.fromNdArrayStreamable(sourceSpec=mnist_train_images_spec)
mnist_train_labels = c3.Dataset.fromNdArrayStreamable(sourceSpec=mnist_train_labels_spec)

In [25]:
# We need to turn labels into one-hot encodings
mnist_test_labels

In [29]:
import sklearn.preprocessing
encoder = sklearn.preprocessing.OneHotEncoder(categories=list(range(10)))
encoder

OneHotEncoder(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [36]:
encoder.categories_ = encoder.categories

In [37]:
encoder.transform(labels)

ValueError: Expected 2D array, got 1D array instead:
array=[3 4 7 2 7 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [20]:
batch_stream.close()

In [14]:
mnist_test_images

c3.Dataset(
 sourceSpec=c3.FileNdArraySourceSpec(
              locationPrefix='http___yann.lecun.com_exdb_mnist_t10k-images-idx3-ubyte.gz',
              storageFormat='NPY',
              optimalFileSize=64000000,
              enableLocalClientStorage=True,
              id='f2b147b8-3585-4d73-ba93-366d16a466ec',
              meta=c3.Meta(
                     tenantTagId=3,
                     tenant='mnistExercise',
                     tag='dev',
                     created=datetime.datetime(2020, 7, 22, 17, 59, 19, tzinfo=datetime.timezone.utc),
                     createdBy='BA',
                     updated=datetime.datetime(2020, 7, 22, 17, 59, 19, tzinfo=datetime.timezone.utc),
                     updatedBy='BA',
                     timestamp=datetime.datetime(2020, 7, 22, 17, 59, 19, tzinfo=datetime.timezone.utc),
                     fetchInclude='[]',
                     fetchType='FileSourceSpec'),
              version=2,
              typeIdent='FISS:FNDASS',
  

In [11]:
mnist_test_labels_onehot = mnist_test_labels.generateDummyVariables(column='0', keepOriginalColumn=False)
mnist_train_labels_onehot = mnist_train_labels.generateDummyVariables(column='0', keepOriginalColumn=False)

500 - NotClassified - c3.stats.machineLearning.AbstractTensorCommon_checkAbstractTensor [1721.343]
message: "Illegal Tensor: 'shape' field must be non-empty."
JSON: {"keepOriginalColumn": false, "column": "0", "this": {"type": "Dataset", "sourceSpec": {"type": "FileNdArraySourceSpec", "locationPrefix": "http___yann.lecun.com_exdb_mnist_t10k-labels-idx1-ubyte.gz", "storageFormat": "NPY", "optimalFileSize": 64000000, "enableLocalClientStorage": true, "id": "a110e642-47a8-42e3-8d5f-92114b902845", "meta": {"type": "Meta", "tenantTagId": 3, "tenant": "mnistExercise", "tag": "dev", "created": "2020-07-22T18:02:39+00:00", "createdBy": "BA", "updated": "2020-07-22T18:02:39+00:00", "updatedBy": "BA", "timestamp": "2020-07-22T18:02:39+00:00", "fetchInclude": "[]", "fetchType": "FileNdArraySourceSpec"}, "version": 2, "typeIdent": "FISS:FNDASS", "location": "http___yann.lecun.com_exdb_mnist_t10k-labels-idx1-ubyte.gz/FileSourceSpec/a110e642-47a8-42e3-8d5f-92114b902845"}}}


C3RuntimeException: 500 - NotClassified - c3.stats.machineLearning.AbstractTensorCommon_checkAbstractTensor [1721.343]
message: "Illegal Tensor: 'shape' field must be non-empty."
JSON: {"keepOriginalColumn": false, "column": "0", "this": {"type": "Dataset", "sourceSpec": {"type": "FileNdArraySourceSpec", "locationPrefix": "http___yann.lecun.com_exdb_mnist_t10k-labels-idx1-ubyte.gz", "storageFormat": "NPY", "optimalFileSize": 64000000, "enableLocalClientStorage": true, "id": "a110e642-47a8-42e3-8d5f-92114b902845", "meta": {"type": "Meta", "tenantTagId": 3, "tenant": "mnistExercise", "tag": "dev", "created": "2020-07-22T18:02:39+00:00", "createdBy": "BA", "updated": "2020-07-22T18:02:39+00:00", "updatedBy": "BA", "timestamp": "2020-07-22T18:02:39+00:00", "fetchInclude": "[]", "fetchType": "FileNdArraySourceSpec"}, "version": 2, "typeIdent": "FISS:FNDASS", "location": "http___yann.lecun.com_exdb_mnist_t10k-labels-idx1-ubyte.gz/FileSourceSpec/a110e642-47a8-42e3-8d5f-92114b902845"}}}

In [8]:
def train_and_test_model(name, model, train_X, train_Y, test_X, test_Y, epochs=10, callbacks=[]):
    pipe = c3.KerasPipe().upsertNativeModel(model=model)
    pipe.technique.numEpochs = epochs
    trained_pipe = pipe.train(input=train_X, targetOutput=train_Y)
    result = trained_pipe.process(input=test_X)

    result_np = result.toNumpyArray()
    test_Y_np = test_Y.toNumpyArray()

    tot = test_Y_np.shape[0]
    error_rate = sum(result_np.argmax(axis=1)!=test_Y_np.argmax(axis=1))/(float(tot))

    print("{} Error Rate: {:.2f}%".format(name, error_rate*100))
    return trained_pipe

In [9]:
# Build tensorflow model.
one_layer_simple_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28*28,), input_shape=(28,28)),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

one_layer_simple_model.summary()
one_layer_simple_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 795,010
Trainable params: 795,010
Non-trainable params: 0
_________________________________________________________________


In [10]:
train_and_test_model('one_layer_simple_model', one_layer_simple_model,
                     mnist_train_images, mnist_train_labels_onehot,
                     mnist_test_images, mnist_test_labels_onehot)

one_layer_simple_model Error Rate: 7.60%


c3.KerasPipe(
 id='48b0e5fa-dd8e-46fa-ada3-5e1999d78255',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 6, 19, 18, 43, 19, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 6, 19, 18, 45, 19, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 6, 19, 18, 45, 19, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_1", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_1", "trainable": true, '
                       '"batch_input_

In [11]:
# Build tensorflow model.
two_layer_simple_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28*28,), input_shape=(28,28)),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

two_layer_simple_model.summary()
two_layer_simple_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_6 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


In [12]:
train_and_test_model('two_layer_simple_model', two_layer_simple_model,
                     mnist_train_images, mnist_train_labels_onehot,
                     mnist_test_images, mnist_test_labels_onehot)

two_layer_simple_model Error Rate: 6.50%


c3.KerasPipe(
 id='4e241871-0d82-4a37-82f3-ec898ddb253d',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 6, 19, 18, 48, 44, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 6, 19, 18, 50, 44, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 6, 19, 18, 50, 44, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_2", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_2", "trainable": true, '
                       '"batch_input_

In [13]:
one_layer_cnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
    tf.keras.layers.Conv2D(32, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

one_layer_cnn_model.summary()
one_layer_cnn_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                216330    
Total params: 216,650
Trainable params: 216,650
Non-trainable params: 0
_________________________________________________________________


In [14]:
train_and_test_model('one_layer_cnn_model', one_layer_cnn_model,
                     mnist_train_images, mnist_train_labels_onehot,
                     mnist_test_images, mnist_test_labels_onehot)

one_layer_cnn_model Error Rate: 5.00%


c3.KerasPipe(
 id='6a4f4628-93eb-402b-a42e-38c7a3db837b',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 6, 19, 18, 54, 11, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 6, 19, 18, 56, 8, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 6, 19, 18, 56, 8, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_3", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_3", "trainable": true, '
                       '"batch_input_sh

In [15]:
three_layer_cnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
    tf.keras.layers.Conv2D(32, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.Conv2D(64, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(125, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

three_layer_cnn_model.summary()
three_layer_cnn_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 125)              

In [16]:
train_and_test_model('three_layer_cnn_model', three_layer_cnn_model,
                     mnist_train_images, mnist_train_labels_onehot,
                     mnist_test_images, mnist_test_labels_onehot)

three_layer_cnn_model Error Rate: 2.89%


c3.KerasPipe(
 id='f105bd4e-65e9-4924-aa75-fa7bd7641ae2',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 6, 19, 18, 59, 32, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 6, 19, 19, 1, 50, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 6, 19, 19, 1, 50, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_4", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_4", "trainable": true, '
                       '"batch_input_sh